<a href="https://colab.research.google.com/github/faiz-aljohani/Arabic-App-Review-Classification/blob/main/ICS472_BERT(1_label).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part-B [65 points]:
 Multiclass and Multilabel App Review Classification

In [1]:
!pip install transformers[torch]
!pip install farasapy
!pip install pyarabic
!pip install arabert
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
!pip install gensim

In [97]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

import keras
from keras import layers, losses, optimizers, metrics
from gensim.models import Word2Vec
from transformers import AutoConfig, BertForSequenceClassification, AutoTokenizer,AutoModelForSequenceClassification, BertModel
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments

import nltk
from nltk import tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# load dataset

In [127]:
dataset = pd.read_csv("https://raw.githubusercontent.com/faiz-aljohani/Arabic-App-Review-Classification/main/AppReview-Multilabel.csv")

In [128]:
dataset

,review,bug_report,improvement_request,rating,others
0,يبيله تصليحات كثير,High,Medium,Low,No
1,ماتحملت ابدددددا ونا دفعت فلوس 🤔😕,No,Medium,Medium,Low
2,اتمنى منكم عمل خيار لتفضيل المسلسل بالكامل عوض...,No,High,Low,No
3,ممتاز وبسيط وتشغيله لا يؤثر على سرعة الانترنت,No,No,High,No
4,تبين م لي ي تم ل يب,No,No,Low,High
...,...,...,...,...,...
2895,بكل حياديه وتجرد انقل تجربتي التطبيق جميل جدا ...,Medium,Medium,Medium,No
2896,مررة صار معفن مدري الا بالقوة بيدخلنا موقعهم ه...,Medium,No,Medium,No
2897,لابد من تطويره ليستفاد منو الجميع,No,High,Low,No
2898,تطبيق اكثر من رائع ومجهود مبارك ،ننتظر المزيد ...,No,Low,High,No


## Split the dataset train/val/test...
70-30 Train TestVal. Take 50% of TestVal data as validation set.
Use seed 777 for partitioning.

### binary labels

In [129]:
def string2number(val):
    mapping = {'High': 3, 'Medium': 2, 'Low': 1, 'No': 0}
    return mapping[val]


def pies_plot(columns, dataset):
    fig, axes = plt.subplots(2, 2, figsize=(6, 6))
    for i, column in enumerate(columns):
        ax = axes[i // 2, i % 2]
        ax.set_title(column)
        ax.pie(dataset[column].value_counts(), labels=dataset[column].unique().tolist(), autopct='%1.1f%%')
    plt.tight_layout()
    plt.show()

### prepare data for multiclass classification

In [130]:

columns_to_convert = ['bug_report', 'improvement_request', 'rating', 'others']
for column in columns_to_convert:
    dataset[column] = dataset[column].apply(string2number)

In [131]:
dataset.head()

,review,bug_report,improvement_request,rating,others
0,يبيله تصليحات كثير,3,2,1,0
1,ماتحملت ابدددددا ونا دفعت فلوس 🤔😕,0,2,2,1
2,اتمنى منكم عمل خيار لتفضيل المسلسل بالكامل عوض...,0,3,1,0
3,ممتاز وبسيط وتشغيله لا يؤثر على سرعة الانترنت,0,0,3,0
4,تبين م لي ي تم ل يب,0,0,1,3


In [132]:
print(dataset.iloc[:,1:].head())

   bug_report  improvement_request  rating  others
0           3                    2       1       0
1           0                    2       2       1
2           0                    3       1       0
3           0                    0       3       0
4           0                    0       1       3


In [133]:

np.random.seed(777)

for index, row in dataset.iterrows():
    max_val = max(row[1:])
    max_indices = [i for i, val in enumerate(row[1:], start=1) if val == max_val]
    selected = np.random.choice(max_indices)
    for i, column in enumerate(dataset.columns[1:], start=1):
        if i == selected:
            dataset.at[index, column] = 1
        else:
            dataset.at[index, column] = 0


In [134]:
dataset.head()

,review,bug_report,improvement_request,rating,others
0,يبيله تصليحات كثير,1,0,0,0
1,ماتحملت ابدددددا ونا دفعت فلوس 🤔😕,0,0,1,0
2,اتمنى منكم عمل خيار لتفضيل المسلسل بالكامل عوض...,0,1,0,0
3,ممتاز وبسيط وتشغيله لا يؤثر على سرعة الانترنت,0,0,1,0
4,تبين م لي ي تم ل يب,0,0,0,1


In [136]:
# Function to convert multi-hot encoded vectors to class indices
def convert_to_class_index(multi_hot_vector):
    return multi_hot_vector.idxmax()

# List of columns to convert
columns_to_convert = ['bug_report', 'improvement_request', 'rating', 'others']

# Apply the conversion function to each specified column
dataset['label'] = dataset.iloc[:,1:].idxmax(axis=1)
dataset.drop(columns_to_convert,axis=1,inplace=True)
# Print the updated DataFrame
# Verify the conversion
print(dataset)

                                                 review                label
0                                    يبيله تصليحات كثير           bug_report
1                     ماتحملت ابدددددا ونا دفعت فلوس 🤔😕               rating
2     اتمنى منكم عمل خيار لتفضيل المسلسل بالكامل عوض...  improvement_request
3         ممتاز وبسيط وتشغيله لا يؤثر على سرعة الانترنت               rating
4                                   تبين م لي ي تم ل يب               others
...                                                 ...                  ...
2895  بكل حياديه وتجرد انقل تجربتي التطبيق جميل جدا ...               rating
2896  مررة صار معفن مدري الا بالقوة بيدخلنا موقعهم ه...           bug_report
2897                  لابد من تطويره ليستفاد منو الجميع  improvement_request
2898  تطبيق اكثر من رائع ومجهود مبارك ،ننتظر المزيد ...               rating
2899  لعبة جميلة جدا اوي تستاهل خمس نجوم التعاليق بس...               rating

[2900 rows x 2 columns]


In [137]:
print(dataset['label'])

0                bug_report
1                    rating
2       improvement_request
3                    rating
4                    others
               ...         
2895                 rating
2896             bug_report
2897    improvement_request
2898                 rating
2899                 rating
Name: label, Length: 2900, dtype: object


In [179]:
id2label = {idx:label for idx, label in enumerate(columns_to_convert)}
label2id = {label:idx for idx, label in enumerate(columns_to_convert)}
print(label2id)

{'bug_report': 0, 'improvement_request': 1, 'rating': 2, 'others': 3}


In [216]:

dataset['label'] = dataset['label'].apply(lambda example: label2id[example])
display(dataset)

,review,label
0,يبيله تصليحات كثير,0
1,ماتحملت ابدددددا ونا دفعت فلوس 🤔😕,2
2,اتمنى منكم عمل خيار لتفضيل المسلسل بالكامل عوض...,1
3,ممتاز وبسيط وتشغيله لا يؤثر على سرعة الانترنت,2
4,تبين م لي ي تم ل يب,3
...,...,...
2895,بكل حياديه وتجرد انقل تجربتي التطبيق جميل جدا ...,2
2896,مررة صار معفن مدري الا بالقوة بيدخلنا موقعهم ه...,0
2897,لابد من تطويره ليستفاد منو الجميع,1
2898,تطبيق اكثر من رائع ومجهود مبارك ،ننتظر المزيد ...,2


### split

In [217]:
X = dataset.iloc[:,0].to_numpy()
y = dataset.iloc[:,1:].to_numpy()

In [218]:
train_sentences, testval_sentences, train_labels, testval_labels = train_test_split(X, y, test_size=0.3, random_state=777)
val_sentences, test_sentences, val_labels, test_labels = train_test_split(testval_sentences, testval_labels, test_size=0.5, random_state=777)

In [219]:
train_labels.shape

(2030, 1)

In [220]:
train_sentences = train_sentences.reshape(-1)
val_sentences = val_sentences.reshape(-1)
test_sentences = test_sentences.reshape(-1)
train_labels = train_labels.reshape(-1)
val_labels = val_labels.reshape(-1)
test_labels = test_labels.reshape(-1)

In [221]:
print(train_sentences.shape)
print(val_sentences.shape)
print(test_sentences.shape)
print(train_sentences.ndim)
print(val_sentences.ndim)
print(test_sentences.ndim)

print(train_labels.ndim)
print(val_labels.ndim)
print(test_labels.ndim)

(2030,)
(435,)
(435,)
1
1
1
1
1
1


## Dataset analysis and report on important statistics.


In [222]:
dataset.head()

,review,label
0,يبيله تصليحات كثير,0
1,ماتحملت ابدددددا ونا دفعت فلوس 🤔😕,2
2,اتمنى منكم عمل خيار لتفضيل المسلسل بالكامل عوض...,1
3,ممتاز وبسيط وتشغيله لا يؤثر على سرعة الانترنت,2
4,تبين م لي ي تم ل يب,3


In [223]:
display(dataset['bug_report'].value_counts())
display(dataset['improvement_request'].value_counts())
display(dataset['rating'].value_counts())
display(dataset['others'].value_counts())

KeyError: 'bug_report'

In [ ]:
columns = ['bug_report', 'improvement_request', 'rating', 'others']
pies_plot(columns, dataset)

1. character count
2. word count
3. max sentence's character and word count
4. mean word length
5. mean sentence length

In [224]:
dataset['char_count'] = dataset['review'].str.len()
dataset['word_count'] = dataset['review'].str.split().str.len()
display(dataset.head())

,review,label,char_count,word_count
0,يبيله تصليحات كثير,0,18,3
1,ماتحملت ابدددددا ونا دفعت فلوس 🤔😕,2,33,6
2,اتمنى منكم عمل خيار لتفضيل المسلسل بالكامل عوض...,1,76,13
3,ممتاز وبسيط وتشغيله لا يؤثر على سرعة الانترنت,2,45,8
4,تبين م لي ي تم ل يب,3,19,7


In [225]:
sortedDataset = dataset.sort_values('char_count', ascending=False)

sortedDataset.head()

,review,label,char_count,word_count
902,من فضلك أعد المحاولة\nمن فضلك أعد المحاولة\nمن...,3,3023,576
87,أكثر من مليار تحميل.... لكن فيه عيوب العيب الأ...,2,500,89
369,العبة جميلة جدا جدا جداجدا جديد جدا حمام الدار...,3,500,100
170,غير متقولوش لعائشة اذا كان علابالي راح نبعتلكم...,3,500,100
1562,جميل جداً ورائع لكن احس في بعض العيوب الليالي ...,3,499,93


In [226]:
dataset['mean_word_length'] = dataset['review'].map(lambda r: np.mean([len(word) for word in r.split()]))
dataset['mean_sent_length'] = dataset['review'].map(lambda r: np.mean([len(sent) for sent in tokenize.sent_tokenize(r)]))
dataset.head()

,review,label,char_count,word_count,mean_word_length,mean_sent_length
0,يبيله تصليحات كثير,0,18,3,5.333333,18.0
1,ماتحملت ابدددددا ونا دفعت فلوس 🤔😕,2,33,6,4.666667,33.0
2,اتمنى منكم عمل خيار لتفضيل المسلسل بالكامل عوض...,1,76,13,4.923077,76.0
3,ممتاز وبسيط وتشغيله لا يؤثر على سرعة الانترنت,2,45,8,4.750000,45.0
4,تبين م لي ي تم ل يب,3,19,7,1.857143,19.0


# Preprocessing

In [227]:
import re

sample = [
    'ماتحملت ابدددددا ونا دفعت الفلوس',
    'ألسلام عليكم في الناس كلب',
    'الََعبة جميلة جدا جدا جداجدا جديد جدا حمام الدار...',
    'ييييي السال',
    'ااا لللل ععع',
    'مما ننا ككا'
]

def custom_preprocessing(sentences):
  sentences_proccesed = []
  for sentence in sentences:
    # Delete جميع الحركات
    sentence = re.sub(r'[\u064B-\u0652]', '', sentence)

    # Delete تحويل (أ, إ) الى ا
    sentence = re.sub(r'[أإ]', 'ا', sentence)

    # Delete حروف الجر, كان واخواتها
    sentence = re.sub(r'\b(فى|في|على|من|الى|كان|لكن)\b', ' ', sentence)

    # Replace ة الى ه
    sentence = re.sub(r'ة', r'ه', sentence)

    # Delete التعريف
    sentence = re.sub(r'\bال', '', sentence)

    # Delete الحرف المكرر اكثر من مرتين بالتوالي
    sentence = re.sub(r'(.)\1{2,}', r'\1\1', sentence)

    # Replace كل حرف يتكرر مرتين الى حرف واحد ما عدا الميم
    sentence = re.sub(r'(?!م|ن)(.)\1', r'\1', sentence)

    # Delete اي حرف لحاله
    sentence = re.sub(r'\b\w\b', '', sentence)

    # Delete اي كلمة اطول من 16 حرف
    sentence = ' '.join(word for word in sentence.split() if len(word) <= 16)

    # Delete اي علامة تتكرر مره وحده بس
    sentence = re.sub(r'([^\w\s])\1', '', sentence)

    sentences_proccesed.append(sentence)

  return sentences_proccesed

processed_samples = custom_preprocessing(sample)
print(processed_samples)

['ماتحملت ابدا ونا دفعت فلوس', 'سلام عليكم ناس كلب', 'عبه جميله جدا جدا جداجدا جديد جدا حمام دار.', 'سال', '', 'مما ننا كا']


In [228]:
df = pd.DataFrame({'Original Sentences': train_sentences, 'Processed Sentences': custom_preprocessing(train_sentences)})

df['proccesed word count'] = df['Processed Sentences'].str.split().apply(len)
df['word count'] = df['Original Sentences'].str.split().apply(len)

df = df.sort_values(by='proccesed word count', ascending=False)

In [229]:
display(df.head(30))

,Original Sentences,Processed Sentences,proccesed word count,word count
1363,من فضلك أعد المحاولة\nمن فضلك أعد المحاولة\nمن...,فضلك اعد محاوله فضلك اعد محاوله فضلك اعد محاول...,432,576
1683,انا كويس الحمد والشكر على كل شيء شي من برا بس ...,انا كويس حمد والشكر كل شيء شي برا بس مو كل شي ...,95,106
192,غير متقولوش لعائشة اذا كان علابالي راح نبعتلكم...,غير متقولوش لعائشه اذا علابالي راح نبعتلكم تصو...,86,100
83,بكل حياديه وتجرد انقل تجربتي التطبيق جميل جدا ...,بكل حياديه وتجرد انقل تجربتي تطبيق جميل جدا جد...,85,92
85,جميل جداً ورائع لكن احس في بعض العيوب الليالي ...,جميل جدا ورائع احس بعض عيوب ليالي بيعلق معي كث...,84,93
289,كل ما افتح مقطع يوتيوب يطلع اعلان\nكل ما افتح ...,كل ما افتح مقطع يوتيوب يطلع اعلان كل ما افتح م...,81,84
924,نصابين ، لاحدا يشتري من المتجر ، بالآخر خسران ...,نصابين ، لاحدا يشتري متجر ، بالآخر خسران قد ما...,79,85
1591,العبة جميلة جدا جدا جداجدا جديد جدا حمام الدار...,عبه جميله جدا جدا جداجدا جديد جدا حمام دار اور...,78,100
1953,والله فرحنا كثيراً بتطبيق مطبعة الملك فهد لكن ...,واله فرحنا كثيرا بتطبيق مطبعه ملك فهد لاسف به ...,77,92
1302,اعزائي الزملاء : بما اني احد منسوبي الصحة ، فه...,اعزائي زملاء : بما اني احد منسوبي صحه ، فهذا ت...,73,76


## Tokenizing

In [230]:
def word_tokenize(docs):
  return [nltk.word_tokenize(doc) for doc in docs]

train_sentences_tokenized = word_tokenize(custom_preprocessing(train_sentences))
val_sentences_tokenized = word_tokenize(custom_preprocessing(val_sentences))
test_sentences_tokenized = word_tokenize(custom_preprocessing(test_sentences))

In [231]:
vocab = set([word for sublist in train_sentences_tokenized for word in sublist])
print(" Number of unique vocab: ", len(vocab))

 Number of unique vocab:  7862


9324 unique vocab without pre-processing

# Models : (Random method will have 25%)

## BERT multiclass classification

### prepare data

In [232]:
train_labels.ndim

1

In [233]:
import pandas as pd

# For training data
train_df = pd.DataFrame({
    'text': train_sentences,
    'label': train_labels,
})

# For validation data
val_df = pd.DataFrame({
    'text': val_sentences,
    'label': val_labels,
})

# For testing data
test_df = pd.DataFrame({
    'text': test_sentences,
    'label': test_labels,
})


In [234]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [235]:
MAX_SEQ_LENGTH = max([len(seq) for seq in train_dataset['text']])
print("Max sequence's length: ", MAX_SEQ_LENGTH)

Max sequence's length:  3023


### Bert tokenizing


In [236]:

model_name = 'aubmindlab/bert-base-arabert'#"google-bert/bert-base-multilingual-uncased"#
num_classes = 4

In [255]:

from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
  # take a batch of texts
#   examples["text"] = word_tokenize(custom_preprocessing(examples["text"]))

  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
#   labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
#   # create numpy array of shape (batch_size, num_labels)
#   labels_matrix = np.zeros((len(text), len(labels)))
#   fill numpy array
#   for idx, label in enumerate(labels):
#     labels_matrix[:, idx] = labels_batch[label]

  encoding["label"] = examples['label']

  return encoding

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Encoding

In [256]:
train_encoded_dataset = train_dataset.map(preprocess_data, batched=True)
val_encoded_dataset = val_dataset.map(preprocess_data, batched=True)
test_encoded_dataset = test_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/2030 [00:00<?, ? examples/s]

Map:   0%|          | 0/435 [00:00<?, ? examples/s]

Map:   0%|          | 0/435 [00:00<?, ? examples/s]

In [259]:
train_encoded_dataset[0]['label']


2

In [260]:
example = train_encoded_dataset[0]
print(example.keys())

dict_keys(['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])


In [261]:
tokenizer.decode(example['input_ids'])

'[CLS] جميله انصح الجميع بتنزيلها واووووو.. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [262]:
example['label']

2

In [265]:
train_encoded_dataset.set_format("torch")
val_encoded_dataset.set_format("torch")
test_encoded_dataset.set_format("torch")

### Build model


In [266]:
# config = AutoConfig.from_pretrained(model_name,num_labels=num_labels, output_attentions=True,problem_type="multi_label_classification")
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=num_classes,
                                                           id2label=id2label,
                                                           label2id=label2id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Training


In [267]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Compute accuracy
    acc = accuracy_score(labels, preds)

    # Compute F1 score (macro average for multi-class classification)
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': acc,
        'f1': f1
    }

In [268]:
batch_size = 8
metric_name = "accuracy"
task = "multilabel-classification"

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # label_names=["bug_report","improvement_request","rating","others" ]
)

In [269]:
print(type(train_dataset[0]["text"]))
print(len(train_dataset))
max_text_length = 0
for sample in train_dataset:
    max_text_length = max(max_text_length, len(sample['text']))

print("Maximum text length:", max_text_length)

<class 'str'>
2030
Maximum text length: 3023


In [272]:
#verify
train_encoded_dataset[0]['label'].type()

'torch.LongTensor'

In [273]:
train_encoded_dataset[0]['input_ids'].shape

torch.Size([128])

In [275]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_encoded_dataset,
    eval_dataset=val_encoded_dataset,
    tokenizer=tokenizer,  # for padding
    compute_metrics=compute_metrics
)

In [276]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.803143,0.678161,0.578762
2,0.869000,0.745915,0.696552,0.603062
3,0.869000,0.905134,0.701149,0.610878
4,0.333500,0.992637,0.717241,0.650922
5,0.333500,1.124290,0.712644,0.640317


TrainOutput(global_step=1270, training_loss=0.5061112381341889, metrics={'train_runtime': 312.3019, 'train_samples_per_second': 32.501, 'train_steps_per_second': 4.067, 'total_flos': 667656291993600.0, 'train_loss': 0.5061112381341889, 'epoch': 5.0})

### Testing ???

In [277]:
trainer.evaluate(test_encoded_dataset)

{'eval_loss': 1.103087067604065,
 'eval_accuracy': 0.6919540229885057,
 'eval_f1': 0.6388716855726913,
 'eval_runtime': 3.0614,
 'eval_samples_per_second': 142.09,
 'eval_steps_per_second': 17.965,
 'epoch': 5.0}

## Proper hyper-parameter tuning based on the validation set

## Classification analysis and possible improvements.


## Final results on the test set.